# Unet modeling

In [1]:
import numpy as np
import os

In [2]:
from PIL import Image

In [3]:
import matplotlib.pyplot as plt

In [8]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms,datasets

# Unet sample
from https://github.com/HyunLee103/Pytorch_practice/tree/master/UNet_regressionhttps://github.com/HyunLee103/Pytorch_practice/tree/master/UNet_regression

## dataset.py

In [14]:
import os
import numpy as np

import torch
import torch.nn as nn

import matplotlib.pyplot as plt
from util import *

## 데이터 로더를 구현하기
class Dataset(torch.utils.data.Dataset):
    def __init__(self, data_dir, transform=None, task=None, opts=None):
        self.data_dir = data_dir
        self.transform = transform
        self.task = task
        self.opts = opts

        # Updated at Apr 5 2020
        self.to_tensor = ToTensor()

        lst_data = os.listdir(self.data_dir)
        lst_data = [f for f in lst_data if f.endswith('jpg') | f.endswith('jpeg') | f.endswith('png')]

        # lst_label = [f for f in lst_data if f.startswith('label')]
        # lst_input = [f for f in lst_data if f.startswith('input')]
        #
        # lst_label.sort()
        # lst_input.sort()
        #
        # self.lst_label = lst_label
        # self.lst_input = lst_input

        lst_data.sort()
        self.lst_data = lst_data

    def __len__(self):
        return len(self.lst_data)

    def __getitem__(self, index):
        # label = np.load(os.path.join(self.data_dir, self.lst_label[index]))
        # input = np.load(os.path.join(self.data_dir, self.lst_input[index]))

        img = plt.imread(os.path.join(self.data_dir, self.lst_data[index]))
        sz = img.shape

        if sz[0] > sz[1]:
            img = img.transpose((1, 0, 2))

        if img.ndim == 2:
            img = img[:, :, np.newaxis]

        if img.dtype == np.uint8:
            img = img / 255.0

        # label = img
        #
        # if self.task == "inpainting":
        #     input = add_sampling(img, type=self.opts[0], opts=self.opts[1])
        # elif self.task == "denoising":
        #     input = add_noise(img, type=self.opts[0], opts=self.opts[1])
        # elif self.task == "super_resolution":
        #     input = add_blur(img, type=self.opts[0], opts=self.opts[1])
        #
        # data = {'input': input, 'label': label}
        #
        # if self.transform:
        #     data = self.transform(data)

        # Updated at Apr 5 2020
        data = {'label': img}

        if self.task == "inpainting":
            data['input'] = add_sampling(data['label'], type=self.opts[0], opts=self.opts[1])
        elif self.task == "denoising":
            data['input'] = add_noise(data['label'], type=self.opts[0], opts=self.opts[1])

        if self.transform:
            data = self.transform(data)

        if self.task == "super_resolution":
            data['input'] = add_blur(data['label'], type=self.opts[0], opts=self.opts[1])

        data = self.to_tensor(data)

        return data

## 트렌스폼 구현하기
class ToTensor(object):
    def __call__(self, data):
        # label, input = data['label'], data['input']
        #
        # label = label.transpose((2, 0, 1)).astype(np.float32)
        # input = input.transpose((2, 0, 1)).astype(np.float32)
        #
        # data = {'label': torch.from_numpy(label), 'input': torch.from_numpy(input)}

        # Updated at Apr 5 2020
        for key, value in data.items():
            value = value.transpose((2, 0, 1)).astype(np.float32)
            data[key] = torch.from_numpy(value)

        return data

class Normalization(object):
    def __init__(self, mean=0.5, std=0.5):
        self.mean = mean
        self.std = std

    def __call__(self, data):
        # label, input = data['label'], data['input']
        #
        # input = (input - self.mean) / self.std
        # label = (label - self.mean) / self.std
        #
        # data = {'label': label, 'input': input}

        # Updated at Apr 5 2020
        for key, value in data.items():
            data[key] = (value - self.mean) / self.std

        return data


class RandomFlip(object):
    def __call__(self, data):
        # label, input = data['label'], data['input']

        if np.random.rand() > 0.5:
            # label = np.fliplr(label)
            # input = np.fliplr(input)

            # Updated at Apr 5 2020
            for key, value in data.items():
                data[key] = np.flip(value, axis=0)

        if np.random.rand() > 0.5:
            # label = np.flipud(label)
            # input = np.flipud(input)

            # Updated at Apr 5 2020
            for key, value in data.items():
                data[key] = np.flip(value, axis=1)

        # data = {'label': label, 'input': input}

        return data


class RandomCrop(object):
  def __init__(self, shape):
      self.shape = shape

  def __call__(self, data):
    # input, label = data['input'], data['label']
    # h, w = input.shape[:2]

    h, w = data['label'].shape[:2]
    new_h, new_w = self.shape

    top = np.random.randint(0, h - new_h)
    left = np.random.randint(0, w - new_w)

    id_y = np.arange(top, top + new_h, 1)[:, np.newaxis]
    id_x = np.arange(left, left + new_w, 1)

    # input = input[id_y, id_x]
    # label = label[id_y, id_x]
    # data = {'label': label, 'input': input}

    # Updated at Apr 5 2020
    for key, value in data.items():
        data[key] = value[id_y, id_x]

    return data

ModuleNotFoundError: No module named 'util'

In [11]:
import os
import numpy as np
import torch
import torch.nn as nn

In [12]:
import tensorflow as tf
from tensorflow import keras

In [13]:
from layer import * # layer에 CBR2d class import

ModuleNotFoundError: No module named 'layer'

In [10]:
## 네트워크 구축
class UNet(nn.Module):
    def __init__(self,nch,nker,norm = 'bnorm',learning_type ='plain'):
        super(UNet, self).__init__()
        self.learning_type = learning_type

        # Contracting path
        self.enc1_1 = CBR2d(in_channels=nch, out_channels=1*nker, norm=norm)
        self.enc1_2 = CBR2d(in_channels=1*nker, out_channels=1*nker,norm=norm)

        self.pool1 = nn.MaxPool2d(kernel_size=2)

        self.enc2_1 = CBR2d(in_channels=1*nker, out_channels=2*nker,norm=norm)
        self.enc2_2 = CBR2d(in_channels=2*nker, out_channels=2*nker,norm=norm)

        self.pool2 = nn.MaxPool2d(kernel_size=2)

        self.enc3_1 = CBR2d(in_channels=2*nker, out_channels=4*nker,norm=norm)
        self.enc3_2 = CBR2d(in_channels=4*nker, out_channels=4*nker,norm=norm)

        self.pool3 = nn.MaxPool2d(kernel_size=2)

        self.enc4_1 = CBR2d(in_channels=4*nker, out_channels=8*nker,norm=norm)
        self.enc4_2 = CBR2d(in_channels=8*nker, out_channels=8*nker,norm=norm)

        self.pool4 = nn.MaxPool2d(kernel_size=2)

        self.enc5_1 = CBR2d(in_channels=8*nker, out_channels=16*nker,norm=norm)

        # Expansive path
        self.dec5_1 = CBR2d(in_channels=16*nker, out_channels=8*nker,norm=norm)

        self.unpool4 = nn.ConvTranspose2d(in_channels=8*nker, out_channels=8*nker,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec4_2 = CBR2d(in_channels=2 * 8*nker, out_channels=8*nker,norm=norm)
        self.dec4_1 = CBR2d(in_channels=8*nker, out_channels=4*nker,norm=norm)

        self.unpool3 = nn.ConvTranspose2d(in_channels=4*nker, out_channels=4*nker,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec3_2 = CBR2d(in_channels=2* 4*nker, out_channels=4*nker,norm=norm)
        self.dec3_1 = CBR2d(in_channels=4*nker, out_channels=2*nker,norm=norm)

        self.unpool2 = nn.ConvTranspose2d(in_channels=2*nker, out_channels=2*nker,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec2_2 = CBR2d(in_channels=2 * 2*nker, out_channels=2*nker,norm=norm)
        self.dec2_1 = CBR2d(in_channels=2*nker, out_channels=1*nker,norm=norm)

        self.unpool1 = nn.ConvTranspose2d(in_channels=1*nker, out_channels=1*nker,
                                          kernel_size=2, stride=2, padding=0, bias=True)

        self.dec1_2 = CBR2d(in_channels=2 * 1*nker, out_channels=1*nker,norm=norm)
        self.dec1_1 = CBR2d(in_channels=1*nker, out_channels=1*nker,norm=norm)

        self.fc = nn.Conv2d(in_channels=1*nker, out_channels=nch, kernel_size=1, stride=1, padding=0, bias=True)

    def forward(self, x):
        enc1_1 = self.enc1_1(x)
        enc1_2 = self.enc1_2(enc1_1)
        pool1 = self.pool1(enc1_2)

        enc2_1 = self.enc2_1(pool1)
        enc2_2 = self.enc2_2(enc2_1)
        pool2 = self.pool2(enc2_2)

        enc3_1 = self.enc3_1(pool2)
        enc3_2 = self.enc3_2(enc3_1)
        pool3 = self.pool3(enc3_2)

        enc4_1 = self.enc4_1(pool3)
        enc4_2 = self.enc4_2(enc4_1)
        pool4 = self.pool4(enc4_2)

        enc5_1 = self.enc5_1(pool4)

        dec5_1 = self.dec5_1(enc5_1)

        unpool4 = self.unpool4(dec5_1)
        cat4 = torch.cat((unpool4, enc4_2), dim=1)
        dec4_2 = self.dec4_2(cat4)
        dec4_1 = self.dec4_1(dec4_2)

        unpool3 = self.unpool3(dec4_1)
        cat3 = torch.cat((unpool3, enc3_2), dim=1)
        dec3_2 = self.dec3_2(cat3)
        dec3_1 = self.dec3_1(dec3_2)

        unpool2 = self.unpool2(dec3_1)
        cat2 = torch.cat((unpool2, enc2_2), dim=1)
        dec2_2 = self.dec2_2(cat2)
        dec2_1 = self.dec2_1(dec2_2)

        unpool1 = self.unpool1(dec2_1)
        cat1 = torch.cat((unpool1, enc1_2), dim=1)
        dec1_2 = self.dec1_2(cat1)
        dec1_1 = self.dec1_1(dec1_2)

        if self.learning_type == 'plain':
            out = self.fc(dec1_1)
        elif self.learning_type == 'residual':
            out = self.fc(dec1_1) + x   # output과 네트워크 인풋을 더해 최종 아웃풋으로 낸다(residual learning)

        return out

ModuleNotFoundError: No module named 'layer'